In [55]:
import itertools
import random
import selenium
from selenium.webdriver import ActionChains
import pyperclip
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException,NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os,time
from newspaper import Article
import pandas as pd
import polars as pl

def extract_text(url):
    '''
    description : The function to extract only main text from article
    :param url: url of the article
    :return: text of article
    '''
    try:
        article = Article(url)
        article.download()
        article.parse()
        text = article.text or np.nan #text가 없는 경우 'N/A'로 출력
    except Exception as e:
        return 'np.nan'
    return text


In [56]:
pd.set_option('display.max_column',None)
pd.set_option('display.max_row',None)

In [23]:
urls_path = '../../data/scraping/'
test = pd.read_csv(urls_path+'articles_urls_0128_2018_4.csv',index_col=0)
test['date'] = pd.to_datetime(test['date'])
test.head(5)

,title,date,url
0,Early images of Netflix.com show how far the s...,2018-04-14,https://qz.com/1248806/early-images-of-netflix...
1,100 Best Netflix Series To Watch Right Now (Ja...,2018-04-06,https://editorial.rottentomatoes.com/guide/bes...
2,Netflix Used Big Data To Identify The Movies T...,2018-04-18,https://www.forbes.com/sites/bernardmarr/2018/...
3,"You can see what Netflix knows about you, but ...",2018-04-17,https://www.usatoday.com/story/tech/columnist/...
4,Why Netflix and Amazon are experimenting with ...,2018-04-27,https://digiday.com/future-of-tv/the-streaming...


In [27]:
urls_path = '../../data/scraping/'
y,m = 2019,4
test = pd.read_csv(urls_path+f'articles_urls_0128_{y}_{m}.csv',index_col=0)
test['date'] = pd.to_datetime(test['date'])
test.head(5)

,title,date,url
0,"Netflix, Amazon and Google Play have 21:9 movi...",2019-04-08,https://www.gsmarena.com/netflix_amazon_and_go...
1,The Obamas and Netflix Just Revealed the Shows...,2019-04-30,https://www.nytimes.com/2019/04/30/business/me...
2,Will Netflix eventually monetize its user data?,2019-04-22,https://theconversation.com/will-netflix-event...
3,"After a long winning streak, Netflix's vulnera...",2019-04-18,https://digiday.com/future-of-tv/netflix-is-ma...
4,Netflix: Barack and Michelle Obama announce de...,2019-05-01,https://www.bbc.com/news/entertainment-arts-48...


In [25]:
articles_df

,title,date,url


In [31]:
years = list(range(2018,2024))
months = [1,4,7,10]
articles_df = pd.read_csv(urls_path+'articles_urls_0128_2018_1.csv',index_col=0)
articles_df['date'] = pd.to_datetime(articles_df['date'])
for year in years:
    for month in months:
        if year == 2018 and month == 1 : continue;
        try: 
            right_join = pd.read_csv(urls_path+f'articles_urls_0128_{year}_{month}.csv',index_col=0)
            right_join['date'] = pd.to_datetime(right_join['date'])
            articles_df = pd.concat([articles_df,right_join],axis=0)
        except:
            print(f"에러난 파일: {year}_{month}.csv")

에러난 파일: 2019_7.csv


In [36]:
articles_2019_7 = pd.read_csv(urls_path+'articles_urls_0128_2019_7_1.csv',index_col=0,encoding_errors='ignore')
articles_2019_7['date'] = pd.to_datetime(articles_2019_7['date'])
articles_df = pd.concat([articles_df, articles_2019_7],axis=0)
articles_df.head(5)

,title,date,url
0,Netflix's history: From DVD rentals to streami...,2018-01-23,https://www.bbc.co.uk/news/newsbeat-42787047
1,'Dark' could be Netflix's biggest European hit...,2018-01-05,https://www.screendaily.com/news/dark-could-be...
2,'Data is everything': Netflix is pouring money...,2018-01-25,https://digiday.com/marketing/data-everything-...
3,Netflix to Release a Series of 15-Minute Stand...,2018-01-22,https://www.vulture.com/2018/01/netflix-15-min...
4,'Open House' Is What Happens When Netflix Give...,2018-01-22,https://www.forbes.com/sites/insertcoin/2018/0...


In [39]:
articles_df_sorted = articles_df.sort_values('date').reset_index().drop(columns='index')
articles_df_sorted.head(3)

,title,date,url
0,Netflix and the 'stolen' indies,2018-01-01,https://www.polygon.com/2018/1/1/16829714/netf...
1,‘The End Of The F***ing World’: Netflix Sets P...,2018-01-01,https://deadline.com/2018/01/the-end-of-the-fi...
2,"What's new on Netflix, Hulu and Amazon in January",2018-01-01,https://abcnews.go.com/Entertainment/netflix-h...


In [48]:
articles_df_sorted['text']=articles_df_sorted['url'].apply(extract_text)

In [52]:
articles_df_sorted['text'].isnull().sum()

0

In [49]:
articles_df_sorted.to_csv('../../data/articles_df_sorted.csv')

In [50]:
articles_df_sorted.head(10)

,title,date,url,text
0,Netflix and the 'stolen' indies,2018-01-01,https://www.polygon.com/2018/1/1/16829714/netf...,Netflix plans to release 80 original films in ...
1,‘The End Of The F***ing World’: Netflix Sets P...,2018-01-01,https://deadline.com/2018/01/the-end-of-the-fi...,Netflix has released the official trailer and ...
2,"What's new on Netflix, Hulu and Amazon in January",2018-01-01,https://abcnews.go.com/Entertainment/netflix-h...,"""Love, Actually,"" ""Breakfast at Tiffany's"" and..."
3,Dave Chappelle addresses Louis C.K. allegation...,2018-01-01,https://ew.com/tv/2018/01/01/dave-chappelle-lo...,In his new Netflix special The Bird Revelation...
4,Netflix surprise drop: 'The End of the ... Wor...,2018-01-01,https://www.usatoday.com/story/life/tv/2018/01...,Lorena Blas\n\nUSA TODAY\n\nTeenage-angst isn'...
5,Sweat: Tobi Pearce and Kayla Itsines' $100m 'N...,2018-01-01,https://www.afr.com/technology/tobi-pearce-and...,"""Netflix, Apple Music, Spotify or whatever, yo..."
6,Netflix Announces 'A Series of Unfortunate Eve...,2018-01-02,https://variety.com/2018/tv/news/netflix-annou...,Netflix has announced that the second season o...
7,‘3%’ is one of Netflix’s biggest shows—but is ...,2018-01-02,https://www.dailydot.com/upstream/3-percent-ne...,Netflix last month released some viewership st...
8,Netflix to release horror movie called ‘The Op...,2018-01-02,https://www.inman.com/2018/01/02/netflix-to-re...,If you weren’t a skeptic of open houses before...
9,"The Best TV Shows and Movies New to Netflix, A...",2018-01-02,https://www.nytimes.com/2018/01/02/watching/ne...,Watching is The New York Times’s TV and film r...


In [53]:
articles_df_sorted.loc[114,'url']

'https://metro.co.uk/2018/01/14/cable-girls-why-you-should-be-watching-the-fantastically-feminist-1920s-netflix-drama-7206102/'

In [54]:
articles_df_sorted.loc[114,'text']

'The second series of Cable Girls is now on Netflix and, for many viewers, it might have passed you by completely.\n\nThis Spanish drama is brave and beautiful, delightful and dark and certainly not one to be missed. It follows a group of women as they deal with the challenges of working as operators for a phone company in 1920s Madrid.\n\nHere’s why you should be watching Cable Girls (Las Chicas del Cable), whether or not you’ve delved into non-English-language shows previously.\n\nThe cast\n\nThe cast are all flawless and each bring something incredibly powerful to their respective roles. The actresses Blanca Suárez, Maggie Civantos, Nadia de Santiago, Ana Fernández and Ana Polvorosa are all sublime.\n\nThere is phenomenal male support, too, thanks to the likes of Yon González, Martiño Rivas, Nico Romero, Borja Luna and Sergio Mur which brings with it a fantastic blend of terrifying, sexy and adorable male characters just as layered and fascinating as the women.\n\nThe feminism\n\nAl

In [ ]:
articles_df_sorted = articles_df_sorted.drop_duplicate()

In [58]:
articles = pd.read_csv('../../data/scraping/articles_df_sorted.csv',index_col=0)

In [59]:
aa = articles.drop_duplicates()
aa.shape

(20498, 4)

In [63]:
pd.read_csv('../../data/scraping/articles_df.csv').shape

(20498, 5)